In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [5]:
text_to_summarize ="""
Sinan Ozdemir is a data scientist, startup founder, and educator living in the San Francisco Bay Area with his dog, 
Charlie; cat, Euclid; and bearded dragon, Fiero. He spent his academic career studying pure mathematics 
at Johns Hopkins University before transitioning to education. He spent several years conducting lectures 
on data science at Johns Hopkins University and at the General Assembly before founding his own startup, 
Kylie.ai, which uses artificial intelligence to build chatbots from historical transcripts. 
After completing a Fellowship at the Y Combinator accelerator, Sinan spent most of his time working on 
his fast-growing company, while creating educational material for data science.
"""


preprocess_text = text_to_summarize.strip().replace("\n","")
t5_prepared_text = "summarize: "+preprocess_text
print ("original text preprocessed:\n", preprocess_text)

input_ids = tokenizer.encode(t5_prepared_text, return_tensors="pt")

input_ids



original text preprocessed:
 Sinan Ozdemir is a data scientist, startup founder, and educator living in the San Francisco Bay Area with his dog, Charlie; cat, Euclid; and bearded dragon, Fiero. He spent his academic career studying pure mathematics at Johns Hopkins University before transitioning to education. He spent several years conducting lectures on data science at Johns Hopkins University and at the General Assembly before founding his own startup, Kylie.ai, which uses artificial intelligence to build chatbots from historical transcripts. After completing a Fellowship at the Y Combinator accelerator, Sinan spent most of his time working on his fast-growing company, while creating educational material for data science.


tensor([[21603,    10,  8271,   152, 15546,   221,  5884,    19,     3,     9,
           331, 17901,     6, 11459,  7174,     6,    11, 23449,   840,    16,
             8,  1051,  5901,  2474,  5690,    28,   112,  1782,     6, 12707,
           117,  1712,     6,  4491, 14758,    26,   117,    11,  4595,   221,
            26, 14580,     6,  3188,    49,    32,     5,   216,  1869,   112,
          2705,  1415,  6908,  4621, 17082,    44,  1079,     7, 24704,   636,
           274,  3508,    53,    12,  1073,     5,   216,  1869,   633,   203,
         13646, 16603,    30,   331,  2056,    44,  1079,     7, 24704,   636,
            11,    44,     8,  2146, 11993,   274, 16072,   112,   293, 11459,
             6, 11478,  1896,     5,     9,    23,     6,    84,  2284,  7353,
          6123,    12,   918,  3582,  4045,     7,    45,  4332, 20146,     7,
             5,   621,     3,  8828,     3,     9, 22172,    44,     8,     3,
           476, 20578,  1016, 30202,     6,  8271,  

In [6]:
# summmarize 
summary_ids = model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=2,
    min_length=30,
    max_length=100,
    early_stopping=True
)

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print (f"Summarized text: \n{output}")

Summarized text: 
Sinan Ozdemir is a data scientist, startup founder, and educator. he founded his own startup, Kylie.ai, which uses artificial intelligence to build chatbots from historical transcripts.


In [7]:
def summarize(text):

    preprocess_text = text.strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text

    input_ids = tokenizer.encode(t5_prepared_Text, return_tensors="pt")
    
    summary_ids = model.generate(
        input_ids,
        num_beams=4,
        no_repeat_ngram_size=2,
        min_length=30,
        max_length=100,
        early_stopping=True
    )

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    return output


In [8]:
summarize(text_to_summarize)

'Sinan Ozdemir is a data scientist, startup founder, and educator. he founded his own startup, Kylie.ai, which uses artificial intelligence to build chatbots from historical transcripts.'

In [9]:
import requests
from random import sample

NYTIMES_KEY = 'uy1HSemASbwBaaSskpdSVckRgSLOVKE6'

results =  requests.get(f'https://api.nytimes.com/svc/mostpopular/v2/viewed/1.json?api-key={NYTIMES_KEY}').json()['results']

for result in results[:5]:
    print(result['title'], result['url'])

article = results[0]

article['title'], article['url']

His Reasons for Opposing Trump Were Biblical. Now a Top Christian Editor Is Out. https://www.nytimes.com/2021/11/14/business/world-magazine-marvin-olasky-trump.html
Alex Jones Loses by Default in Remaining Sandy Hook Defamation Suits https://www.nytimes.com/2021/11/15/us/politics/alex-jones-sandy-hook.html
Biden signs infrastructure bill, promoting benefits for Americans. https://www.nytimes.com/2021/11/15/us/politics/biden-signs-infrastructure-bill.html
Petra Mayer, Books Editor for NPR and ‘Resident Nerd,’ Dies at 46 https://www.nytimes.com/2021/11/14/us/npr-petra-mayer-dead.html
Will Real Estate Ever Be Normal Again? https://www.nytimes.com/2021/11/12/magazine/real-estate-pandemic.html


('His Reasons for Opposing Trump Were Biblical. Now a Top Christian Editor Is Out.',
 'https://www.nytimes.com/2021/11/14/business/world-magazine-marvin-olasky-trump.html')

In [10]:
from bs4 import BeautifulSoup

article_body = BeautifulSoup(requests.get(article['url']).text).find('section', {'name': 'articleBody'}).text

print(article_body)

When Marvin Olasky gets angry emails from readers — more often than not about an exposé of wrongdoing at an evangelical church, or about a story that reflects poorly on Donald Trump — he has a stock reply.“We think this is useful to the Church,” he tells disgruntled readers, “because we are also sinners.”As the longtime editor of World, a Christian news organization that has a website, a biweekly magazine and a set of podcasts, Mr. Olasky has delivered a mix of hard news and watchdog articles about the evangelical realm under a journalistic philosophy he calls “biblical objectivity.”It involves taking strong stands where the Bible is clear, which has led World to oppose abortion rights and support refugees, he says, and to follow reportable facts where the Bible doesn’t provide clear guidance.The concept served Mr. Olasky well from 1994, when he became the editor of World, until Nov. 1, 2021, when he submitted his resignation.He had, he said, received an effective “vote of no confidenc

In [11]:
def naive_sliding_window(text):

    estimated_char_per_word = 5
    windows_needed = max(1, round(len(text) / estimated_char_per_word / 512))
    print(f'Sliding over {windows_needed} windows')
    window_size = 512 * estimated_char_per_word

    for window in range(windows_needed):
        yield text[window*window_size: (window + 1)*window_size]
        

In [12]:
for article_window in naive_sliding_window(article_body):  # put chunks that fit the 512 max tokens
    print(summarize(article_window))
    print()
    

Token indices sequence length is longer than the specified maximum sequence length for this model (594 > 512). Running this sequence through the model will result in indexing errors


Sliding over 4 windows
john avlon: marvin Olasky's departure is another example of the american news media sinking deeper into polarization. he says the publication has shaken evangelical churches and related institutions with its deeply reported articles.

a few longtime staff members have left over the last year. one journalist whose departure particularly rattled the newsroom is Mindy Belz, who resigned in October.

many conservative evangelical leaders have gotten behind Mr. Trump. john avlon says he's been criticized for running articles by doomsday olasky, whose views aren't rooted in christianity espoused in the church tv show "the savior"

the future of World’s newsroom is unclear, but much of its online real estate is devoted to the new opinion section. editor is Mr. Mohler, who opened it with a promise to join "the battle of ideas"



In [13]:
article_body.count('\n')

0

In [14]:
input_ids = tokenizer.encode('translate English to German: Where is the chocolate?', return_tensors='pt')

# summmarize 
translate_ids = model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print (f"Translated text:\n{output}")


Translated text:
Wo ist die Schokolade?


In [15]:
input_ids = tokenizer('translate English to German: Where is the chocolate?', return_tensors='pt').input_ids
labels = tokenizer('Wo ist die Schokolade?', return_tensors='pt').input_ids

loss = model(input_ids=input_ids, labels=labels).loss

labels, loss

(tensor([[ 3488,   229,    67, 31267,    58,     1]]),
 tensor(0.1136, grad_fn=<NllLossBackward>))

In [16]:
input_ids = tokenizer.encode('cola sentence: Where is the chocolate?', return_tensors='pt')

# summmarize 
translate_ids = model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"is grammatically correct?: \n{output}")


is grammatically correct?: 
acceptable


In [17]:
input_ids = tokenizer.encode('cola sentence: Where be a chocolates?', return_tensors='pt')

# summmarize 
translate_ids = model.generate(
    input_ids,
    max_length=20,
    early_stopping=True
)

output = tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"is grammatically correct?: \n{output}")

is grammatically correct?: 
unacceptable


In [18]:
sentence_one = 'How to fish'
sentence_two = 'Fishing Manual for beginnners'


input_ids = tokenizer.encode(f"stsb sentence1: {sentence_one} sentence2: {sentence_two}", return_tensors='pt')

# summmarize 
translate_ids = model.generate(
    input_ids,
    max_length=3,
    early_stopping=True
)

output = tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"semantically similar? (1-5): \n{output}")



semantically similar? (1-5): 
3.2


In [19]:
sentence_one = 'How to fish'
sentence_two = 'Hiking Manual for beginnners'


input_ids = tokenizer.encode(f"stsb sentence1: {sentence_one} sentence2: {sentence_two}", return_tensors='pt')

# summmarize 
translate_ids = model.generate(
    input_ids,
    max_length=3,
    early_stopping=True
)

output = tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"semantically similar? (1-5): \n{output}")



semantically similar? (1-5): 
0.4
